# IMPORTS

In [130]:
import re
import requests
import pandas as pd
import numpy as np
import math
from bs4 import BeautifulSoup
from datetime import datetime

# DATA COLLECT

In [131]:

#PARAMENTS
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}


#link princial do modelo de roupa que pŕeciso
url= 'https://www2.hm.com/en_us/men/products/jeans.html'

#REQUEST TO URL
page = requests.get( url, headers=headers )


#BEAUTIFUL OBJECT
soup = BeautifulSoup(page.text, 'html.parser')

# ================================ PRODUCT DATA =========== # 

products = soup.find( 'ul', class_='products-listing small' )


#list of products
products_list= products.find_all ('article',class_="hm-product-item")


#product id

product_id = [p.get('data-articlecode') for p in products_list]

#product category

products_category= [p.get('data-category') for p in products_list]

#product name 

#extraction all the name 
products_list_2=  products.find_all('a', class_='link')

products_name =[p.get_text('link') for p in products_list_2]

#product price

#extraction informations of price
products_list_3 = products.find_all('span', class_='price regular')

#find all the price in page
products_price= [p.get_text('price_regular') for p in products_list_3]

data = pd.DataFrame([product_id,products_category,products_name,products_price]).T

data.columns = ['product_id','products_category','products_name','products_price']

data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d-%H-%M:%S')

In [132]:
data.shape

(36, 5)

In [4]:
data.head()

,product_id,products_category,products_name,products_price,scrapy_datetime
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-05-12-15-23:02
1,1008549006,men_jeans_regular,Regular Jeans,$ 19.99,2022-05-12-15-23:02
2,1004199004,men_jeans_skinny,Skinny Cropped Jeans,$ 29.99,2022-05-12-15-23:02
3,0985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,2022-05-12-15-23:02
4,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-05-12-15-23:02


# DATA  COLLECT BY PRODUCT

In [191]:
#key browser
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

#empty Data Frame (to insert the joins)
df_compositions= pd.DataFrame()

aux= []

cols= ['Composition','Size','Art. No.','Fit','messages.garmentLength','messages.waistRise',
'Care instructions',
 'Material',
 'Description',
 'Imported',
 'Concept',
 'Nice to know']
    
df_pattern = pd.DataFrame( columns=  cols )

#df_pattern = pd.DataFrame( columns=['Size','Art. No.','Composition'] )


#API Requests
for i in range(len(data)):
    
    url= 'https://www2.hm.com/en_us/productpage.'+ data.loc[i,'product_id'] +'.html'
    #url= 'https://www2.hm.com/en_us/productpage.'+ '1024256001' +'.html'
    
    #print('Product:{}'.format(url))
    
    
    page = requests.get( url, headers=headers )

    #Beautifull Souo object
    soup= BeautifulSoup(page.text,'html.parser')

    ### ===================== color name ======================

    products_list= soup.find_all('a',class_='filter-option miniature active') + soup.find_all('a',class_='filter-option miniature') 

    #product id
    products_id = [p.get('data-articlecode') for p in products_list]

    #color name
    color_name = [p.get('data-color') for p in products_list]

    #create data frame

    df_color = pd.DataFrame([products_id,color_name]).T

    #rename the columns of DF
    df_color.columns= ['Art. No.','color_name']
    
    
    for j in range(len(df_color)):
        url2= 'https://www2.hm.com/en_us/productpage.'+ df_color.loc[j,'Art. No.'] +'.html'
        #print( 'Color: {}'.format( url2 ) )
        
    
        page = requests.get( url2, headers=headers )

        #Beautifull Souo object
        soup= BeautifulSoup(page.text,'html.parser')
        
        #=========================product name ==============================
        product_name = soup.find_all('h1',class_='primary product-item-headline')
        product_name = product_name[0].get_text()
        
        # ==================== product price ======================= #
        
        product_price= soup.find_all('div',class_='primary-row product-item-price')
        product_price = re.findall(r'\d+?.\d+',product_price[0].get_text())[0]
       
        
        
        #create new colum styly_id
        #df_color['stlye_id'] = df_color['product_id'].apply(lambda x: x[:-3])

        #create new colum color_id
        #df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

        # ============================== composition =============================== #

        #composition
        composition_list= soup.find_all('div', class_ = 'details-attributes-list-item')
        
        #composition_list = soup.find_all('dd','details-list-item')
        #product_composition = composition_list[3]

        product_composition = [list(filter (None, p.get_text().split('\n') ) ) for p in composition_list]
        

        if product_composition != []:

            #created the dataframe aux
            df_composition= pd.DataFrame(product_composition).T
        
            
            #renome the columns
            df_composition.columns= df_composition.iloc[0]

            #delete the first row and delete the columns that is the empty
            df_composition= df_composition.iloc[1:].fillna(method='ffill')

            
            
            #select columns 
            #df_composition= df_composition[['Composition','Art. No.']]

            #delete the first row and delete the columns that is the empty
            #df_composition= df_aux.iloc[1:3]

            #copying the code and the model to row the below
            #df_composition = df_composition.fillna(method='ffill')


            #rename the data frame to a new variable
            #df_composition = df_composition.copy(True)

            #remove pocket lining, shell and lining
            df_composition['Composition'] = df_composition['Composition'].replace('Pocket lining:','',regex=True)
            df_composition['Composition'] = df_composition['Composition'].replace('Shell:','',regex=True)
            df_composition['Composition'] = df_composition['Composition'].replace('Lining:','',regex=True)
        
       

    
            # garantee the same number of columns
            df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

            #rename the columns 
            #df_composition.columns = ['composition', 'size', 'art. no.', 'fit', 'messages.garmentLength',
            #'messages.waistRise', 'care instructions', 'material', 'description',
            #'imported', 'concept', 'nice to know','more sustainable materials']

            #create new colum styly_id
            #df_composition['stlye_id'] =df_composition['Art. No.'].apply(lambda x: x[:-3])

            #create new colum color_id
            #df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

            #keep new columns if it shows
            aux= aux + df_composition.columns.tolist()

    
            #merge dataframe of color and composition 
            df_composition= pd.merge(df_composition,df_color, how='left', on ='Art. No.')
            


            #all products 
            df_compositions = pd.concat([df_composition,df_compositions], axis=0)

        
        
    else:
        None

#select just columns necessary
df_compositions= df_compositions[['Art. No.','Composition','color_name','Size','Fit']]

#rename columns
df_compositions.rename(columns = {'Art. No.':'product_id','Composition':'composition','Size':'size','Fit':'fit'},inplace=True)

# Join Showroom data + details

df_compositions['stlye_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )

df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d-%H-%M:%S')



# DATA CLEANING

In [ ]:
#loading the data
data = pd.read_csv('/home/guilherme/Documentos/repos/ds_ao_dev/projeto_webscraping/df_raw.csv')

#delete the columns not necessery
data = data.drop(['Unnamed: 0' ],axis=1)

#product id

data = data.dropna(subset=['product_id'])

#product category

#product name
data['products_name']= data['products_name'].apply(lambda x: x.replace(' ','_').lower())
#products_price

#já fiz o tratamento anteriomente

#product datetime


data['scrapy_datetime']= pd.to_datetime(data['scrapy_datetime'])

#style id

#color id

#color name
data['color_name']= data['color_name'].apply(lambda x: x.replace(' ','_').replace('/','_').lower() if pd.notnull(x) else x)

#Composition

data['Composition']= data['Composition'].where(~data['Composition'].str.contains('Pocket lining:',na=False) ) 

data['Composition']= data['Composition'].where(lambda x: ~x.str.contains('Shell:',na=False) )

data['Composition'] = data['Composition'].where(lambda x: ~x.str.contains('Lining:',na=False) )

#drop duplicates
#first were making drop the that are duplicates (excect the colum Composition), because the colums is duplicate
data= data.drop_duplicates(subset= ['product_id', 'products_category', 'products_name', 'products_price',
       'scrapy_datetime', 'stlye_id', 'color_id', 'color_name'],keep='last')

   
#reset index
data = data.reset_index(drop=True)


#break composition by comma (separando as composições dos materiais por coluna)
df1 = data['Composition'].str.split(',',expand=True)

#Cotton| Spandex | Polyester |  Elastomultiester
#2 etapa
#definir as colunas (organizando as colunas por máteria-prima)

#criando Dataframe vazio com o mesmo tamanho do dataframe original para prencher com as respectivas composições
df_ref = pd.DataFrame(index=np.arange(len(data) ),columns=['cotton','spandex','polyester','elastomultiester'])

#cotton
#pegando a coluna da composição cotton
df_cotton = df1[0]

#colocando o nome na coluna 
df_cotton.name = 'cotton'
#mesclando com a tabela vazia 
df_ref = pd.concat([df_ref,df_cotton],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['cotton'] = df_ref['cotton'].fillna( 'Cotton 0%' )

#polyester 

#adicionando as coluna da composição Polyester
df_polyester =df1[1].where(lambda x: x.str.contains('Polyester',na=True))

#renomeando a coluna
df_polyester.name= 'polyester'


df_ref= pd.concat([df_ref,df_polyester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['polyester'] = df_ref['polyester'].fillna( 'Polyester 0%' )

#Elastomultiester

#coletando apenas a composição elastano
df_elastomultiester =df1[1].where(lambda x: x.str.contains('Elastomultiester',na=True))

#renomenado a coluna eslastano
df_elastomultiester.name= 'elastomultiester'


#mesclando com df_ref vazio e com a mesma quantidade de linhas que há no index
df_ref= pd.concat([df_ref,df_elastomultiester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['elastomultiester'] = df_ref['elastomultiester'].fillna( 'Elastomultiester 0%' )

#Spandex

#coletando composição spandex da segunda coluna df1

df_spandex= df1[1].where(lambda x: x.str.contains('Spandex',na=True))

df_spandex.name = 'spandex'

#combine from spandex botth columns 1 and 2

df_spandex = df_spandex.combine_first(df1[2])


df_ref= pd.concat([df_ref,df_spandex],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

df_ref['spandex'] = df_ref['spandex'].fillna('Spandex 0%')

#coletando composição spandex da terceira coluna do df1

#df_spandex_2= df1[2].where(lambda x: x.str.contains('Spandex',na=True))
             
#df_spandex_2.name= 'spandex'

#df_ref= pd.concat([df_ref,df_spandex_2],axis=1)

#df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]



#final join

data= pd.concat([data,df_ref],axis=1)

#coletando apenas os numeros e transformando em números inteiros

#format composition data

#cotton
data['cotton']= data['cotton'].apply(lambda x: int(re.search('\d+',x ).group(0))/100 if pd.notnull(x) else x)

#spandex
data['spandex'] =data['spandex'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

#polyester
data['polyester']= data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#elastomultiester
data['elastomultiester']=data['elastomultiester'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

#drop columns
data= data.drop(['Composition'],axis=1)

#drop duplicates


data= data.drop_duplicates()

data.shape